In [ ]:
### First cell, read your database, here I convert the DNA sequences from letters to int's.

#importando o arquivo, base de dados
import os
os.chdir('C:\\Users\\Usuario\\Desktop\\dna')
data1= open('splice.data.txt', 'r')

#passando o arquivo para um list, com as classes convertidas para int, converte as series para int
f= data1.readlines()
for i in range(len(f)):
    f[i]= f[i].split(',')
for i in range(len(f)):
    del(f[i][1])
for i in range(len(f)):
    f[i][1]=f[i][1].split()[0]
c=[]
for i in range(len(f)):
    b=[]
    if f[i][0]=='EI':
        b.append(0)
    elif f[i][0]=='IE':
        b.append(1)
    elif f[i][0]=='N':
        b.append(2)
    a=[]
    for j in range(len(f[i][1])):
        if f[i][1][j]=='A':
            a.append(0)
        elif f[i][1][j]=='C':
            a.append(1)
        elif f[i][1][j]=='G':
            a.append(2)
        elif f[i][1][j]=='T':
            a.append(3)
        else:
            a.append(8)
    b.append(a)
    c.append(b)
cdata=[]
ctarg=[]

for i in range(len(c)):
    cdata.append(c[i][1])
    ctarg.append((c[i][0]))

In [ ]:
### Import tools
#importando ferramentas
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn import tree
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier


from sklearn.feature_selection import SelectFromModel

#ensemble
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
### Count how many elements each class has.
#contando quantos integrantes de cada classe na base de dados
j1=0
j2=0
j3=0
for i in range(len(ctarg)):
    if(ctarg[i]==0):
        j1=j1+1
    elif(ctarg[i]==1):
        j2=j2+1
    elif(ctarg[i]==2):
        j3=j3+1
print(j1,j2,j3)

In [ ]:
### Separate the base for each class, then creat esemble models (80% of database is for training, 10% for weight definition, 10% for test, done 10 times) like
### a voting classifier but each opinion of a algorithm has a weight based on the success it had during the weight definition.

#Voting Classifier with weight, pesos definidos pela media da porcentagem de acerto de cada classe em uma 10-fold cross-validation
#primeiro separa a base em 90% treino,  10% para testes, de maneira que se repita 10 vezes, tendo testes para toda a base
class0 = cdata[0:767]
class1 = cdata[767:1535]
class2 = cdata[1535:3190]
class0T = ctarg[0:767]
class1T = ctarg[767:1535]
class2T = ctarg[1535:3190]

clf1 = SVC(gamma='auto')
clf2 = KNeighborsClassifier(n_neighbors=2)
clf3 = tree.DecisionTreeClassifier()
clf4 = GaussianNB()
VotingWithWeightsPredictions=[]
predictions0= []
predictions1= []
predictions2= []
i=0
while i<10:
    #separacao da base
    dataTeste= class0[int(i*len(class0)/10):int((i+1)*len(class0)/10)]+class1[int(i*len(class1)/10):int((i+1)*len(class1)/10)]+class2[int(i*len(class2)/10):int((i+1)*len(class2)/10)]
    c0= class0.copy()
    c1= class1.copy()
    c2= class2.copy()
    del(c0[int(i*len(class0)/10):int((i+1)*len(class0)/10)])
    del(c1[int(i*len(class1)/10):int((i+1)*len(class1)/10)])
    del(c2[int(i*len(class2)/10):int((i+1)*len(class2)/10)])
    data= c0 + c1 + c2
    targTeste= class0T[int(i*len(class0T)/10):int((i+1)*len(class0T)/10)]+class1T[int(i*len(class1T)/10):int((i+1)*len(class1T)/10)]+class2T[int(i*len(class2T)/10):int((i+1)*len(class2T)/10)]
    c0T= class0T.copy()
    c1T= class1T.copy()
    c2T= class2T.copy()
    del(c0T[int(i*len(class0T)/10):int((i+1)*len(class0T)/10)])
    del(c1T[int(i*len(class1T)/10):int((i+1)*len(class1T)/10)])
    del(c2T[int(i*len(class2T)/10):int((i+1)*len(class2T)/10)])
    targ= c0T + c1T + c2T
    
    #calculando os pesos, 10-fold cross-validation no conjunto de treino apenas
    j=0
    M1W= []      #list para armezar os pesos do modelo 1
    M2W= []
    M3W= []
    M4W= []
    while j<10:
        #Separacao do conjunto de treino em 10-fold 
        dataPTeste =c0[int(j*len(c0)/10):int((j+1)*len(c0)/10)]+c1[int(j*len(c1)/10):int((j+1)*len(c1)/10)]+c2[int(j*len(c2)/10):int((j+1)*len(c2)/10)]
        c0P= c0.copy()
        c1P= c1.copy()
        c2P= c2.copy()
        del(c0P[int(j*len(c0)/10):int((j+1)*len(c0)/10)])
        del(c1P[int(j*len(c1)/10):int((j+1)*len(c1)/10)])
        del(c2P[int(j*len(c2)/10):int((j+1)*len(c2)/10)])
        dataP = c0P + c1P + c2P
        targPTeste =c0T[int(j*len(c0T)/10):int((j+1)*len(c0T)/10)]+c1T[int(j*len(c1T)/10):int((j+1)*len(c1T)/10)]+c2T[int(j*len(c2T)/10):int((j+1)*len(c2T)/10)]
        c0TP= c0T.copy()
        c1TP= c1T.copy()
        c2TP= c2T.copy()
        del(c0TP[int(j*len(c0T)/10):int((j+1)*len(c0T)/10)])
        del(c1TP[int(j*len(c1T)/10):int((j+1)*len(c1T)/10)])
        del(c2TP[int(j*len(c2T)/10):int((j+1)*len(c2T)/10)])
        targP = c0TP + c1TP + c2TP
        
        #construcao dos modelos, predicao do teste
        clf1.fit(dataP,targP)
        clf2.fit(dataP,targP)
        clf3.fit(dataP,targP)
        clf4.fit(dataP,targP)
        
        T1=clf1.predict(dataPTeste)
        T2=clf2.predict(dataPTeste)
        T3=clf3.predict(dataPTeste)
        T4=clf4.predict(dataPTeste)
        
        #confusion matrix e calculo dos pesos por porcentagem de acerto dos palpites
        cm1 = confusion_matrix(targPTeste, T1)
        cm2 = confusion_matrix(targPTeste, T2)
        cm3 = confusion_matrix(targPTeste, T3)
        cm4 = confusion_matrix(targPTeste, T4)
        
        T1weights= [cm1[0][0]/(cm1[0][0] + cm1[1][0] + cm1[2][0]), cm1[1][1]/(cm1[0][1] + cm1[1][1] + cm1[2][1]), cm1[2][2]/(cm1[0][2] + cm1[1][2] + cm1[2][2])]
        T2weights= [cm2[0][0]/(cm2[0][0] + cm2[1][0] + cm2[2][0]), cm2[1][1]/(cm2[0][1] + cm2[1][1] + cm2[2][1]), cm2[2][2]/(cm2[0][2] + cm2[1][2] + cm2[2][2])]
        T3weights= [cm3[0][0]/(cm3[0][0] + cm3[1][0] + cm3[2][0]), cm3[1][1]/(cm3[0][1] + cm3[1][1] + cm3[2][1]), cm3[2][2]/(cm3[0][2] + cm3[1][2] + cm3[2][2])]
        T4weights= [cm4[0][0]/(cm4[0][0] + cm4[1][0] + cm4[2][0]), cm4[1][1]/(cm4[0][1] + cm4[1][1] + cm4[2][1]), cm4[2][2]/(cm4[0][2] + cm4[1][2] + cm4[2][2])]
        
        M1W.append(T1weights)
        M2W.append(T2weights)
        M3W.append(T3weights)
        M4W.append(T4weights)
        
        j= j+1
    #media dos pesos de cada modelo
    M1Wm =[0,0,0]
    M2Wm =[0,0,0]
    M3Wm =[0,0,0]
    M4Wm =[0,0,0]
    for v in range(len(M1W)):
        M1Wm[0] += M1W[v][0]
        M1Wm[1] += M1W[v][1]
        M1Wm[2] += M1W[v][2]
        M2Wm[0] += M2W[v][0]
        M2Wm[1] += M2W[v][1]
        M2Wm[2] += M2W[v][2]
        M3Wm[0] += M3W[v][0]
        M3Wm[1] += M3W[v][1]
        M3Wm[2] += M3W[v][2]
        M4Wm[0] += M4W[v][0]
        M4Wm[1] += M4W[v][1]
        M4Wm[2] += M4W[v][2]
    M1Wm= [M1Wm[0]/10, M1Wm[1]/10, M1Wm[2]/10]
    M2Wm= [M2Wm[0]/10, M2Wm[1]/10, M2Wm[2]/10]
    M3Wm= [M3Wm[0]/10, M3Wm[1]/10, M3Wm[2]/10]
    M4Wm= [M4Wm[0]/10, M4Wm[1]/10, M4Wm[2]/10]
    
    #fit model e votacao
    clf1.fit(data,targ)
    clf2.fit(data,targ)
    clf3.fit(data,targ)
    clf4.fit(data,targ)

    T1=clf1.predict(dataTeste)
    T2=clf2.predict(dataTeste)
    T3=clf3.predict(dataTeste)
    T4=clf4.predict(dataTeste)
    
    
    
    for k in range(len(targTeste)):
        predictClass= [0,0,0]
        predictClass[T1[k]]= predictClass[T1[k]] + M1Wm[T1[k]]
        predictClass[T2[k]]= predictClass[T2[k]] + M2Wm[T2[k]]
        predictClass[T3[k]]= predictClass[T3[k]] + M3Wm[T3[k]]
        predictClass[T4[k]]= predictClass[T4[k]] + M4Wm[T4[k]]
        if(targTeste[k]==0):
            predictions0.append(np.argmax(predictClass))
        if(targTeste[k]==1):
            predictions1.append(np.argmax(predictClass))
        if(targTeste[k]==2):
            predictions2.append(np.argmax(predictClass))
    
    i= i+1
VotingWithWeightsPredictions= predictions0 + predictions1 + predictions2
    
#Verificando a precisao
c0=0
p0=0
c1=0
p1=0
c2=0
p2=0
for j in range(len(ctarg)):
        if(ctarg[j]==0):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p0=p0+1
            c0= c0 +1
        if(ctarg[j]==1):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p1=p1+1
            c1= c1 +1
        if(ctarg[j]==2):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p2=p2+1
            c2= c2 +1

print("Accuracy: %0.2f  , ei: %0.2f , ie: %0.2f, n: %0.2f" % ((p0/c0+ p1/c1 + p2/c2)/3, p0/c0, p1/c1, p2/c2))
conf_mat = confusion_matrix(ctarg, VotingWithWeightsPredictions)
print('Confusion:\n', conf_mat, '\n')